In [1]:
%config IPCompleter.greedy=True
import numpy as np
import cv2
import random
import operator
import math
import tensorflow as tf

In [2]:
class Card:
    def __init__(self):
        
        self.contour = []    #card contour
        self.width = 0       #contour max width?
        self.height = 0      #contour max height?
        self.center = []
        self.coords = []     #the rects 4 corners
        self.warped = []     #warped img of the card
        self.warped_thresh = []
        
        self.num_img = []
        self.suit_img = []
        
        self.num = None
        self.suit = None
        
    def draw_warped(self):
        cv2.imshow(self.num+self.suit+str(self.height)+str(self.width), self.warped)
        
    def draw_card_outlines(self, img):
        cv2.polylines(img, np.array([self.coords]), True, (0,255,255), 2)
        
    def draw_suit(self):
        cv2.imshow(str(np.sum(self.suit_img)), self.suit_img)
        
    def draw_num(self):
        cv2.imshow(str(np.sum(self.num_img)), self.num_img)
        
    def draw_text(self, img):
    
        suit_text = ["Hearts", "Spades", "Diamonds", "Clubs"]
        num_text = ['Ace','Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack','Queen','King']
    
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        TEXT = num_text[self.num-1] + " " + suit_text[self.suit]
        FONT_SIZE = 0.8
        FONT_COLOR = (0, 255, 0)
        THICKNESS = 2
        
        ######
        
        # get boundary of this text
        textsize = cv2.getTextSize(TEXT, FONT, FONT_SIZE, THICKNESS)[0]

        # get coords based on boundary
        text_x = int(self.center[0] - (textsize[0]/2))
        text_y = int(self.center[1] + (textsize[1]/2))
        
        text_y = text_y - int(self.height/2) - 25
        
        
        # Add text centered on image
        cv2.putText(img, TEXT, (text_x, text_y), FONT, FONT_SIZE, FONT_COLOR, thickness=THICKNESS)
    
    def set_center(self):
        
        # Adds x's together & y's together and takes the avg
        pts_sum = np.sum(self.coords, axis=0)/len(self.coords)
        
        # Lav til int fordi vi ikke kan lande mellem 2 pixels
        x = int(pts_sum[0])
        y = int(pts_sum[1])
        
        self.center = [x, y]
        
        
    def find_warped_card(self, img):
        
        order_rect = lambda p: math.atan2(p[1]-self.center[1], p[0]-self.center[0])
        
        self.coords = sorted(self.coords, key=order_rect)
        
        w, h = 200, 300 # width, height

        # lav firkant i det format vi vil warp til
        dst = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], dtype = "float32")
        
        ###################################

        # Lav første firkant ud fra self.coords og konverter til float
        rect1 = np.array(self.coords, dtype = "float32").reshape((4,2))
        
        # Compute the perspective transformation matrix
        transformation_matrix = cv2.getPerspectiveTransform(rect1, dst)
        
        # Lav første warp
        warp1 = cv2.warpPerspective(img, transformation_matrix, (w, h))
        
        ###################################
        
        # Anden firkant er samme firkant som før bare startende fra et nabo-punkt
        rect2 = np.array(self.coords[1:] + [self.coords[0]], dtype = "float32").reshape((4,2))
        
        # Compute the perspective transformation matrix
        transformation_matrix = cv2.getPerspectiveTransform(rect2, dst)
        
        # Lav anden warp
        warp2 = cv2.warpPerspective(img, transformation_matrix, (w, h))
        
        ####################################
        
        # Sæt self.warped til den warp som har den mindste værdi i det øverste venstre hjørne.
        # Den warp der har mindst hvidt (og derfor lavest værdi) burde være den, der vender den tigtige vej.
        self.warped = min(warp1, warp2, key=lambda x: np.sum(x[12:75, 9:39]))
        
        
    def find_icons(self):
        
        # FOUND BY SLAVE LABOR
        CORNER_WIDTH = 48
        CORNER_HEIGHT = 140
        
        # Taken from comparison imgs
        NUM_WIDTH = 70
        NUM_HEIGHT = 125
        SUIT_WIDTH = 70
        SUIT_HEIGHT = 100

        #####
        
        # 1.Cut corner out 2. Make it bigger 3. Make it grayscale
        corner = self.warped[0:CORNER_HEIGHT, 0:CORNER_WIDTH]
        corner_big = cv2.resize(corner, (0,0), fx=3, fy=3)
        corner_big_gray = cv2.cvtColor(corner_big, cv2.COLOR_BGR2GRAY)
        

        # Make it black & white and more crisp
        thresh = cv2.threshold(corner_big_gray, 180, 255, cv2.THRESH_BINARY_INV)[1]
        #thresh = cv2.adaptiveThreshold(corner_big_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 333, 1)
        
        self.warped_thresh = thresh
        
        # Divide img into number & suit
        num  = thresh[0:255, 0:144]
        suit = thresh[256:420, 0:144]
        
        self.num_img  = self.crop_and_resize_icon(num, NUM_WIDTH, NUM_HEIGHT)
        self.suit_img = self.crop_and_resize_icon(suit, SUIT_WIDTH, SUIT_HEIGHT)
        
        self.find_num()  # find numerisk værdi fra icon
        self.find_suit() # find numerisk værdi fra icon
        
    def crop_and_resize_icon(self, img, new_width, new_height):
    
        #find contours
        contours = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]

        # Check om der blev fundet noget icon
        if(len(contours) == 0): return 

        # Find det icon med det største areal
        icon = max(contours, key=cv2.contourArea)

        # Find afgrænsende firkant
        x, y, w, h = cv2.boundingRect(icon)

        # Crop icon
        rect = img[y:y+h, x:x+w]

        #return resized icon
        return cv2.resize(rect, (new_width, new_height), 0, 0)
    
    def find_num(self):
        
        # lav et dict med key=filnavn og value=img
        num_img_dict = {x: cv2.imread("./test_cards/"+str(x)+".jpg", cv2.IMREAD_GRAYSCALE) for x in range(1,14)}
        
        # lav nyt dict med key=filnavn og value=absdiff (jo mindre et tal jo bedre et match)
        best_match_dict = {k: np.sum(cv2.absdiff(self.num_img, v)) for k, v in num_img_dict.items()}
        
        # sætter self.num til det bedste match
        self.num = min(best_match_dict, key=best_match_dict.get)

        
    def find_suit(self):
        
        suits = ["h", "s", "d", "c"]
        
        # lav et dict med key=filnavn og value=img
        suit_img_dict = {x:cv2.imread("./test_cards/"+x+".jpg", cv2.IMREAD_GRAYSCALE) for x in suits}

        # lav nyt dict med key=filnavn og value=absdiff (jo mindre et tal jo bedre et match)
        best_match_dict = {k: np.sum(cv2.absdiff(self.suit_img, v)) for k, v in suit_img_dict.items()}
        
        # sætter self.suit til det bedste match
        self.suit = suits.index(min(best_match_dict, key=best_match_dict.get))
        
        
    def validate(self): return self.num is not None and self.suit is not None
        
            

In [3]:
def find_cards(og_frame, pp_frame):
    
    #List holding valid cards
    found_cards = []
    
    #Find all contours from given frame
    contours = cv2.findContours(pp_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    #Go through each contour as c
    for c in contours:
        
        size = cv2.contourArea(c)                       # areal
        peri = cv2.arcLength(c, True)                   # omkreds
        approx = cv2.approxPolyDP(c, 0.01*peri, True)   # reducere antal hjørner -> prøv at sæt den op til 0.05 (var 0.01)
    
        is_rectangle = len(approx) == 4                 # har 4 hjørner/punkter(vertices)
        is_reasonable_size = size>4000 and size<100000  # er firkanten en realistisk størrelse
    
    
        if(is_rectangle and is_reasonable_size):        # Prøv at lav figuren til et kort
            
            card = Card()
        
            card.contour = c
            
            card.coords = list(approx.reshape(4,2))
            
            card.width, card.height = cv2.boundingRect(c)[2:]
            
            card.set_center()
            
            card.find_warped_card(og_frame) # Find og extract kortet fra billedet og sæt det i self.warped 

            card.find_icons()
        
            if card.validate(): found_cards.append(card) # Hvis kortet er validt. Tilføj til found_cards
                
    return found_cards

def draw_predicted_hand(img, text):

    FONT = cv2.FONT_HERSHEY_SIMPLEX
    TEXT = text
    FONT_SIZE = 2
    FONT_COLOR = (255, 0, 255)
    THICKNESS = 5

    ######

    # get boundary of this text
    text_w, text_h = cv2.getTextSize(TEXT, FONT, FONT_SIZE, THICKNESS)[0]
    
    img_h, img_w, _ = img.shape
    
    text_x = int(img_w/2 - text_w/2)
    text_y = int(img_h - text_h)

    cv2.putText(img, TEXT, (text_x, text_y), FONT, FONT_SIZE, FONT_COLOR, thickness=THICKNESS)

    
def draw_warps_and_icons_from_cards(cards):
    
    # [Height,Width,Depth]
    s = [1080,1920,3]
    
    if(len(cards)) == 0: return np.zeros([540,1920,3], dtype=np.uint8)
    
    
    res1 = np.zeros((cards[0].warped_thresh.shape[0],0 ), dtype=np.uint8) #makes an empty 420 x 0 img to make the card in cards loop easier

    for card in cards[:5]:
        res1 = np.hstack((res1, card.warped_thresh))
    
    size_scale = s[0] / 2 / res1.shape[0] 
    res1 = cv2.resize(res1, (0,0), fx=size_scale, fy=size_scale )
    res1 = np.hstack((np.zeros((res1.shape[0], s[1]//2-res1.shape[1]), dtype=np.uint8), res1))
    res1 = cv2.cvtColor(res1, cv2.COLOR_GRAY2RGB)
        

    res2 = np.zeros((540,0,3), dtype=np.uint8)
    for card in cards[:5]:
        res2 = np.hstack((res2, cv2.resize(card.warped, (192,540))))
        
    res2 = np.hstack((res2,np.zeros((540, 960 - res2.shape[1], 3), dtype=np.uint8)))
    
    return np.hstack((res1,res2))
#     cv2.imshow("warps_thresh", res1)
#     cv2.imshow("warps", res2)


In [4]:

def preprocess_frame(frame):
    
    threshold = 160
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.threshold(blur,threshold,255,cv2.THRESH_BINARY)[1]
    return thresh

def preprocess_frame_clahe(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
    cl1 = clahe.apply(gray)
    thresh = cv2.threshold(cl1, 160 ,255,cv2.THRESH_BINARY)[1]
    return thresh

def preprocess_frame_adap(frame):
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gaus = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 33, 1)
    return gaus



def find_cards_from_image(frame, model=None): # returns new frame & list of all cards found
    
    # prepross_frame = preprocess_frame(frame) 
    # prepross_frame = preprocess_frame_clahe(frame)
    prepross_frame = preprocess_frame_adap(frame)
    
    found_cards = find_cards(frame, prepross_frame) # returns a list of cards from the processed frame
    
    for card in found_cards:
        
        card.draw_card_outlines(frame) # outliner kortet
        card.draw_text(frame)          # tegner suit osv. over kortet

    
    ###############################
    if model is not None:
        
        text = str(len(found_cards)) + "/5 cards found"

        if len(found_cards) >= 5:
            nums = [card.num for card in found_cards[:5]]
            suits = [card.suit for card in found_cards[:5]]
            arr = [[nums + suits]]
            
            num_dict = {0:"Nothing",1:"Pair",2:"Two Pair",3:"Three of a kind",4:"Straight",5:"Flush",6:"Full House",7:"Four of a kind",8:"Straight Flush",9:"Royal Flush"}
            
            prediction = np.argmax(model.predict_proba(arr))
            text = num_dict[prediction]

        draw_predicted_hand(frame, text)
    ##############################
    
    warps_img = draw_warps_and_icons_from_cards(found_cards)
    res = np.hstack((cv2.cvtColor(prepross_frame,cv2.COLOR_GRAY2RGB),frame))
    res = cv2.resize(res, (1920,int(1920*0.5625*0.5)))
    res = np.vstack((res,warps_img))
    res = cv2.resize(res, (0,0), fx=0.8, fy=0.8)
    
    return found_cards, res  

In [5]:
def save_img_icon(img_path, save_path, suit=True, num=True):
    
    img = cv2.imread(img_path)
    
    prepross_frame = preprocess_frame(img) # processes frame to make it easier to find cards
    
    found_cards = find_cards(img, prepross_frame) # returns a list of found cards
    
    if(len(found_cards) == 0): return
    
    card = found_cards[0] # take first card (there should only be one)
    
    if suit: cv2.imwrite(save_path, card.suit_img)
    if num: cv2.imwrite(save_path, card.num_img)

In [6]:
def use_img(img):
    img = cv2.imread(img)
    model = tf.keras.models.load_model("models/hupra_desktop_model_100.h5")
    
    cards, preview = find_cards_from_image(img, model)
    
#     cv2.namedWindow("Feed", cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty("Feed", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
#     cv2.namedWindow("Feed2", cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty("Feed2", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    cv2.imshow("Feed", img)
    cv2.imshow("Feed2", preview)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
def use_cam(num=0):
    cap = cv2.VideoCapture(num)
    
    model = tf.keras.models.load_model("models/hupra_desktop_model_100.h5")

    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'): break
        ret, frame = cap.read()
        cards, preview = find_cards_from_image(frame, model)
        
        cv2.imshow("Feed", frame)
        cv2.imshow("Feed2", preview)

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(0)

In [8]:
use_img("imgs/img8.jpg")
#use_cam(1)
#save_img_icon("./pre/clubs.jpg", "./delete/this2.jpg", suit=False)

Instructions for updating:
Colocations handled automatically by placer.
5
4
2
3
1


In [9]:
num_imgs = {str(x):cv2.imread("./test_cards/"+str(x)+".jpg", cv2.IMREAD_GRAYSCALE) for x in range(1,14)}
# cv2.imshow("Feed", num_imgs[13])
# cv2.waitKey(0)
# cv2.destroyAllWindows()